# Notebook 2: Conditional Routing

## Learning Objectives
- Understand conditional edges in LangGraph
- Build graphs with dynamic routing based on state
- Implement intent-based routing patterns

## Why Conditional Routing?

Real-world applications need to make decisions:
- Route to different handlers based on user intent
- Loop back when more information is needed
- Skip steps when they're not necessary

LangGraph's **conditional edges** enable this dynamic behavior.

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

from typing import Literal, Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from IPython.display import Image, display

llm = ChatOpenAI(model="gpt-5-mini", temperature=0)

## Example 1: Simple Router

Let's build a support bot that routes to different specialists based on the query type.

In [ ]:
# Define state
class SupportState(TypedDict):
    messages: Annotated[list, add_messages]
    query_type: str  # Will be set by the router

# Router function - determines which specialist to use
def classify_query(state: SupportState) -> dict:
    """Classify the user's query into a category"""
    last_message = state["messages"][-1].content
    
    response = llm.invoke([
        SystemMessage(content="""Classify the user query into one of these categories:
        - billing: Questions about payments, invoices, subscriptions
        - technical: Technical issues, bugs, how-to questions
        - general: Everything else
        
        Respond with ONLY the category name (billing, technical, or general)."""),
        HumanMessage(content=last_message)
    ])
    
    query_type = response.content.strip().lower()
    return {"query_type": query_type}

# Specialist handlers
def billing_specialist(state: SupportState) -> dict:
    """Handle billing-related queries"""
    response = llm.invoke([
        SystemMessage(content="You are a billing specialist. Help with payment and subscription issues. Be concise."),
        *state["messages"]
    ])
    return {"messages": [response]}

def technical_specialist(state: SupportState) -> dict:
    """Handle technical queries"""
    response = llm.invoke([
        SystemMessage(content="You are a technical support specialist. Help with bugs and technical issues. Be concise."),
        *state["messages"]
    ])
    return {"messages": [response]}

def general_specialist(state: SupportState) -> dict:
    """Handle general queries"""
    response = llm.invoke([
        SystemMessage(content="You are a helpful customer support agent. Be friendly and concise."),
        *state["messages"]
    ])
    return {"messages": [response]}

In [ ]:
# The routing function - returns the name of the next node
def route_to_specialist(state: SupportState) -> Literal["billing", "technical", "general"]:
    """Route to the appropriate specialist based on query_type"""
    query_type = state.get("query_type", "general")
    
    if query_type == "billing":
        return "billing"
    elif query_type == "technical":
        return "technical"
    else:
        return "general"

# Build the graph
builder = StateGraph(SupportState)

# Add nodes
builder.add_node("classify", classify_query)
builder.add_node("billing", billing_specialist)
builder.add_node("technical", technical_specialist)
builder.add_node("general", general_specialist)

# Add edges
builder.add_edge(START, "classify")

# Add conditional edge from classifier
builder.add_conditional_edges(
    "classify",  # Source node
    route_to_specialist,  # Function that decides the route
    {  # Mapping of return values to node names
        "billing": "billing",
        "technical": "technical",
        "general": "general"
    }
)

# All specialists go to END
builder.add_edge("billing", END)
builder.add_edge("technical", END)
builder.add_edge("general", END)

support_graph = builder.compile()

# Visualize
display(Image(support_graph.get_graph().draw_mermaid_png()))

In [ ]:
# Test with different query types
queries = [
    "Why was I charged twice this month?",
    "The app keeps crashing when I try to upload a file",
    "What are your business hours?"
]

for query in queries:
    result = support_graph.invoke({
        "messages": [HumanMessage(content=query)],
        "query_type": ""
    })
    print(f"Query: {query}")
    print(f"Routed to: {result['query_type']}")
    print(f"Response: {result['messages'][-1].content}")
    print("-" * 50)

## Example 2: Looping with Conditions

A powerful pattern is looping until a condition is met. Let's build a fact-checker that keeps refining until confident.

In [ ]:
class FactCheckState(TypedDict):
    claim: str
    analysis: str
    confidence: float
    iterations: int

def analyze_claim(state: FactCheckState) -> dict:
    """Analyze the claim and assess confidence"""
    response = llm.invoke([
        SystemMessage(content="""Analyze this claim for accuracy. Provide:
        1. Your analysis
        2. A confidence score from 0.0 to 1.0. 0.0 meaning the claim is totally inaccurate, 1.0 meaning the claim is accurate.
        
        Format your response as:
        ANALYSIS: <your analysis>
        CONFIDENCE: <score>"""),
        HumanMessage(content=f"Claim: {state['claim']}\n\nPrevious analysis: {state.get('analysis', 'None')}")
    ])
    
    content = response.content
    
    # Parse the response
    analysis = content.split("CONFIDENCE:")[0].replace("ANALYSIS:", "").strip()
    try:
        confidence = float(content.split("CONFIDENCE:")[1].strip())
    except:
        confidence = 0.5
    
    return {
        "analysis": analysis,
        "confidence": confidence,
        "iterations": state["iterations"] + 1
    }

def should_continue(state: FactCheckState) -> Literal["continue", "done"]:
    """Decide whether to continue analyzing or finish"""
    # Stop if confident enough or max iterations reached
    if state["confidence"] >= 0.8 or state["iterations"] >= 3:
        return "done"
    return "continue"

# Build the graph
builder = StateGraph(FactCheckState)

builder.add_node("analyze", analyze_claim)

builder.add_edge(START, "analyze")

# Conditional edge that can loop back
builder.add_conditional_edges(
    "analyze",
    should_continue,
    {
        "continue": "analyze",  # Loop back!
        "done": END
    }
)

fact_checker = builder.compile()

display(Image(fact_checker.get_graph().draw_mermaid_png()))

In [ ]:
# Test the fact checker
result = fact_checker.invoke({
    "claim": "The Great Wall of China is visible from space with the naked eye.",
    "analysis": "",
    "confidence": 0.0,
    "iterations": 0
})

print(f"Claim: {result['claim']}")
print(f"Final Analysis: {result['analysis']}")
print(f"Confidence: {result['confidence']}")
print(f"Iterations: {result['iterations']}")

## Pattern: Using LLM for Routing Decisions

Instead of hard-coded rules, you can use an LLM to make routing decisions. This is especially useful for complex, nuanced decisions.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

# Define structured output for routing
class RouteDecision(BaseModel):
    """Decision on how to route the user's request"""
    route: Literal["search", "calculate", "chat"] = Field(
        description="The route to take: search for web queries, calculate for math, chat for conversation"
    )
    reasoning: str = Field(description="Brief explanation of why this route was chosen")

# Create a structured LLM
structured_llm = llm.with_structured_output(RouteDecision)

class RouterState(TypedDict):
    messages: Annotated[list, add_messages]
    route: str
    reasoning: str

def llm_router(state: RouterState) -> dict:
    """Use LLM to decide the route"""
    last_message = state["messages"][-1].content
    
    decision = structured_llm.invoke([
        SystemMessage(content="Decide how to handle this user request."),
        HumanMessage(content=last_message)
    ])
    
    return {
        "route": decision.route,
        "reasoning": decision.reasoning
    }

def handle_search(state: RouterState) -> dict:
    return {"messages": [AIMessage(content="[Search handler] I would search the web for: " + state["messages"][-1].content)]}

def handle_calculate(state: RouterState) -> dict:
    return {"messages": [AIMessage(content="[Calculator] I would calculate: " + state["messages"][-1].content)]}

def handle_chat(state: RouterState) -> dict:
    response = llm.invoke(state["messages"])
    return {"messages": [response]}

def get_route(state: RouterState) -> str:
    return state["route"]

# Build graph
builder = StateGraph(RouterState)

builder.add_node("router", llm_router)
builder.add_node("search", handle_search)
builder.add_node("calculate", handle_calculate)
builder.add_node("chat", handle_chat)

builder.add_edge(START, "router")
builder.add_conditional_edges("router", get_route, ["search", "calculate", "chat"])
builder.add_edge("search", END)
builder.add_edge("calculate", END)
builder.add_edge("chat", END)

smart_router = builder.compile()

display(Image(smart_router.get_graph().draw_mermaid_png()))

In [ ]:
# Test the smart router
test_queries = [
    "What's the capital of France?",
    "What is 234 * 567?",
    "Hello, how are you today?"
]

for query in test_queries:
    result = smart_router.invoke({
        "messages": [HumanMessage(content=query)],
        "route": "",
        "reasoning": ""
    })
    print(f"Query: {query}")
    print(f"Route: {result['route']} (Reason: {result['reasoning']})")
    print(f"Response: {result['messages'][-1].content}")
    print("-" * 50)

---

## Exercise 2: Build a Content Moderator

Create a graph that:
1. Takes user input
2. Classifies it as "safe", "warning", or "blocked"
3. Routes to appropriate handlers:
   - **safe**: Respond normally
   - **warning**: Respond with a gentle reminder about guidelines
   - **blocked**: Refuse to respond

**Bonus**: Add a loop that asks for clarification if the content is ambiguous.

In [ ]:
# YOUR CODE HERE

class ModerationState(TypedDict):
    messages: Annotated[list, add_messages]
    classification: str

# TODO: Define classifier node

# TODO: Define handler nodes (safe_handler, warning_handler, blocked_handler)

# TODO: Define routing function

# TODO: Build the graph

# Test
# result = moderation_graph.invoke({...})

### Exercise 2: Solution (hidden)

<details>
<summary>Click to reveal solution</summary>

```python
class ModerationState(TypedDict):
    messages: Annotated[list, add_messages]
    classification: str

def classify_content(state: ModerationState) -> dict:
    last_message = state["messages"][-1].content
    response = llm.invoke([
        SystemMessage(content="""Classify this content:
        - safe: Normal, appropriate content
        - warning: Borderline content that needs a reminder
        - blocked: Inappropriate content that should be refused
        
        Respond with ONLY: safe, warning, or blocked"""),
        HumanMessage(content=last_message)
    ])
    return {"classification": response.content.strip().lower()}

def safe_handler(state: ModerationState) -> dict:
    response = llm.invoke(state["messages"])
    return {"messages": [response]}

def warning_handler(state: ModerationState) -> dict:
    response = llm.invoke([
        SystemMessage(content="Respond helpfully but include a gentle reminder about community guidelines."),
        *state["messages"]
    ])
    return {"messages": [response]}

def blocked_handler(state: ModerationState) -> dict:
    return {"messages": [AIMessage(content="I'm sorry, but I can't help with that request.")]}

def route_by_classification(state: ModerationState) -> str:
    c = state.get("classification", "safe")
    if c == "warning":
        return "warning"
    elif c == "blocked":
        return "blocked"
    return "safe"

builder = StateGraph(ModerationState)
builder.add_node("classify", classify_content)
builder.add_node("safe", safe_handler)
builder.add_node("warning", warning_handler)
builder.add_node("blocked", blocked_handler)

builder.add_edge(START, "classify")
builder.add_conditional_edges("classify", route_by_classification, ["safe", "warning", "blocked"])
builder.add_edge("safe", END)
builder.add_edge("warning", END)
builder.add_edge("blocked", END)

moderation_graph = builder.compile()
```
</details>

## Key Takeaways

1. **Conditional edges** enable dynamic routing based on state
2. **Router functions** return the name of the next node
3. You can **loop** by pointing an edge back to a previous node
4. **Structured output** makes LLM-based routing more reliable
5. Always include **exit conditions** for loops to prevent infinite execution

## Next: Tool Integration

In the next notebook, we'll learn how to give our agents tools to interact with the world!